In [1]:
import re

TOKEN_RE = [
    ('NUMBER', r'\d+'),
    ('ID', r'[a-zA-Z_][a-zA-Z0-9_]*'),
    ('PLUS', r'\+'),
    ('MINUS', r'-'),
    ('TIMES', r'\*'),
    ('DIVIDE', r'/'),
    ('LPAREN', r'\('),
    ('RPAREN', r'\)'),
    ('TRUE', r'true'),
    ('FALSE', r'false'),
    ('SKIP', r'[ \t\n]+'),
    ('MISMATCH', r'.')
]

def tokenize(code):
    tokens = []
    while code:
        match = None
        for token_type, token_regex in TOKEN_RE:
            regex = re.compile(token_regex)
            match = regex.match(code)
            if match:
                if token_type != 'SKIP':
                    tokens.append((token_type, match.group(0)))
                code = code[match.end(0):]
                break
        if not match:
            raise SyntaxError(f'Неизвестный символ: {code[0]}')
    return tokens

In [2]:
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_token = None
        self.token_index = -1
        self.next_token()

    def next_token(self):
        self.token_index += 1
        if self.token_index < len(self.tokens):
            self.current_token = self.tokens[self.token_index]
        else:
            self.current_token = None

    def parse(self):
        return self.E()

    def E(self):
        node = self.T()
        while self.current_token and self.current_token[0] in ('PLUS', 'MINUS'):
            op = self.current_token
            self.next_token()
            node = (op, node, self.T())
        return node

    def T(self):
        node = self.F()
        while self.current_token and self.current_token[0] in ('TIMES', 'DIVIDE'):
            op = self.current_token
            self.next_token()
            node = (op, node, self.F())
        return node

    def F(self):
        if self.current_token[0] == 'LPAREN':
            self.next_token()
            node = self.E()
            self.next_token()
            return node
        elif self.current_token[0] == 'ID':
            node = self.current_token
            self.next_token()
            return node
        elif self.current_token[0] == 'NUMBER':
            node = self.current_token
            self.next_token()
            return node
        elif self.current_token[0] in ('TRUE', 'FALSE'):
            node = self.current_token
            self.next_token()
            return node
        raise SyntaxError('Ожидался фактор')

def main():
    code = "a + b * 3 - true"
    tokens = tokenize(code)
    parser = Parser(tokens)
    parse_tree = parser.parse()
    print('Разобранное выражение:', parse_tree)

if __name__ == "__main__":
    main()

Разобранное выражение: (('MINUS', '-'), (('PLUS', '+'), ('ID', 'a'), (('TIMES', '*'), ('ID', 'b'), ('NUMBER', '3'))), ('ID', 'true'))
